In [ ]:
!pip install polars
!pip install pandas
!pip install seaborn

In [ ]:
import polars as pl
import pandas as pd

# Read only the first 1000 rows
chartevents = pl.read_csv(
    './datasets/KMIMIC/chartevents.csv',
)
chartevents = chartevents.with_columns(pl.col("charttime").str.to_datetime("%Y-%m-%d %H:%M:%S"))
chartevents = chartevents.with_columns(pl.col("storetime").str.to_datetime("%Y-%m-%d %H:%M:%S"))

In [ ]:
diagnoses_icd = pd.read_csv('./datasets/KMIMIC/diagnoses_icd.csv')

# 

j80 = diagnoses_icd[
    (diagnoses_icd['icd_code'].str.startswith('J960')) |
    (diagnoses_icd['icd_code'].str.startswith('J961')) |
    (diagnoses_icd['icd_code'].str.startswith('J962')) |
    (diagnoses_icd['icd_code'].str.startswith('J969'))
]
j80_hadm = j80['hadm_id'].unique()

In [ ]:
import polars as pl
labevents = pl.read_csv('datasets/KMIMIC/labevents.csv')
labevents_po2 = labevents.filter(pl.col('itemid').is_in(['###########', '###########']))
labevents_po2_under_60 = labevents_po2.filter(pl.col("valuenum") < 60)
labevents_soa2 = labevents.filter(pl.col('itemid').is_in(['###########', '###########']))
labevents_sao2_under_90 = labevents_soa2.filter(pl.col("valuenum") < 90)
po2_hadm = labevents_po2_under_60['hadm_id'].unique()
sao2_hadm = labevents_sao2_under_90['hadm_id'].unique()

In [ ]:
all_vals = list(set(j80_hadm.tolist() + po2_hadm.to_list() + sao2_hadm.to_list()))
all_vals

In [ ]:
# E(eye)/V(verbal)/M(motor) > Glasgow coma scale(verbal)
calculation_table = chartevents.filter(pl.col('item_id') == "#############").filter(pl.col('hadm_id').is_in(all_vals))
# .filter(pl.col('value').is_in(['E', 'T']))
calculation_table = calculation_table.unique()
calculation_table

In [ ]:
results = []

for hid in all_vals:
    a = calculation_table.filter(pl.col('hadm_id') == hid).sort('charttime')
    if a.is_empty():
        continue

    # Calculate durations
    a = a.with_columns([
        pl.col('charttime').shift(1).alias("last_measurement")
    ])
    
    a = a.with_columns([
        (pl.col('charttime') - pl.col('last_measurement')).alias("duration")
    ])

    # Extract all 'E' charttimes
    first_time = a.filter(pl.col('value') == 'E').sort('charttime')['charttime']
    if len(first_time) == 0:
        continue

    # First and last intubation
    first_time_a = first_time[0]
    last_time_a = first_time[-1]

    # Cumulative vent duration (native duration object)
    cumulative_vent_duration = (
        a.filter(pl.col('value') == 'E')
         .select(pl.col('duration').sum().alias('vent_duration'))
         .get_column("vent_duration")[0]
    )

    # First trach
    trach_date = a.filter(pl.col('value') == 'T').sort('charttime')['charttime']
    trach_date_a = trach_date[0] if len(trach_date) > 0 else None

    # ============================================
    # NEW: Check for any E–E gap > 8 hrs with 1-5 between
    # ============================================
    a_rows = a.select(['value', 'charttime']).rows()
    
    prev_e_time = None
    found_intermediate = False
    gap_gt_8hrs_with_1_5 = False

    for val, time in a_rows:
        if val == 'E':
            if prev_e_time and found_intermediate:
                gap = time - prev_e_time
                if gap.total_seconds() > 8 * 3600:
                    gap_gt_8hrs_with_1_5 = True
                    break
            prev_e_time = time
            found_intermediate = False
        elif val in {'1', '2', '3', '4', '5'} and prev_e_time:
            found_intermediate = True
    # ============================================

    results.append({
        'hadm_id': hid,
        'first_intubation_time': first_time_a,
        'last_intubation_time': last_time_a,
        'cumulative_vent_duration': cumulative_vent_duration,
        'first_trach_time': trach_date_a,
        'gap_gt_8hrs_with_1_5': gap_gt_8hrs_with_1_5  # <=== NEW FIELD
    })

results_df = pl.DataFrame(results)
results_df = results_df.with_columns([
    pl.col("first_trach_time").is_not_null().alias("trach_flag")
])
results_df = results_df.filter((pl.col('first_trach_time').is_null() | (pl.col('cumulative_vent_duration') != 0)) & ~pl.col('gap_gt_8hrs_with_1_5'))
results_df = results_df.drop("first_trach_time", "gap_gt_8hrs_with_1_5")
results_df

In [ ]:
final_tab = results_df
labevents_po2 = labevents.filter(pl.col('itemid').is_in(['001L00053', '001L301013']))

lpo2 = labevents_po2.group_by('hadm_id').agg(pl.col('valuenum').min())
lpo2 = lpo2.rename({'valuenum': 'min_po2'})
lpo2

sao2 = labevents_soa2.group_by('hadm_id').agg(pl.col('valuenum').min())
sao2 = sao2.rename({'valuenum': 'min_so2'})
sao2

final_tab = final_tab.join(lpo2, on='hadm_id', how='left')
final_tab = final_tab.join(sao2, on='hadm_id', how='left')
final_tab
# exclusion criteria
patients = pl.read_csv('datasets/KMIMIC/admissions.csv')[['hadm_id', 'deathtime']]
patients = patients.with_columns(pl.col("deathtime").str.to_datetime("%Y.%m.%d %H:%M"))
final_tab = final_tab.join(patients, on="hadm_id", how="left")
final_tab = final_tab.with_columns(
    (pl.col('deathtime') - pl.col('first_intubation_time')).alias('death_minus_first')
)
final_tab = final_tab.filter(pl.col('death_minus_first').is_null() | (pl.col('death_minus_first') > pl.duration(days=3))).filter(pl.col('cumulative_vent_duration') > pl.duration(days=3))
final_tab = final_tab.drop('deathtime', 'death_minus_first')

In [ ]:
import pandas as pd
# searching d_items
d_labitems = pd.read_csv('datasets/KMIMIC/d_items.csv')
d_labitems[d_labitems['label'].str.contains('tidal volume', case=False)]

In [ ]:
ventilation_df = final_tab

for vital_name, item_id in vitals.items():
    if not item_id:
        continue  # skip entries with empty item_id

    # infer window based on ID type
    if item_id.startswith("001C_"):
        early_window_hours = 2
    elif item_id.startswith("001L"):
        early_window_hours = 12
    else:
        continue  # skip unknown format

    # Join and filter chartevents for this item_id
    joined = (
        chartevents
        .filter(pl.col("item_id") == item_id)
        .join(ventilation_df.select(["hadm_id", "first_intubation_time"]), on="hadm_id", how="inner")
    )

    # EARLY window: ± early_window_hours (4 hrs after)
    result_early = (
        joined
        .with_columns([
            (pl.col("first_intubation_time") + pl.duration(hours=4) - pl.duration(hours=early_window_hours)).alias("start_window"),
            (pl.col("first_intubation_time") + pl.duration(hours=4) + pl.duration(hours=early_window_hours)).alias("end_window"),
            pl.col("first_intubation_time").alias("exact_point_early")
        ])
        .filter(
            pl.col("charttime").is_between(pl.col("start_window"), pl.col("end_window"))
        )
        .with_columns([
            (pl.col("charttime") - pl.col("exact_point_early")).dt.total_seconds().abs().alias("abs_diff_secs")
        ])
    )

    closest_early = (
        result_early
        .sort(["hadm_id", "abs_diff_secs"])
        .group_by("hadm_id")
        .first()
        .select(["hadm_id", "valuenum"])
        .rename({"valuenum": f"{vital_name}_4h"})
    )

    # 72-HOUR window: always ±1 hour
    result_72 = (
        joined
        .with_columns([
            (pl.col("first_intubation_time") + pl.duration(hours=72) - pl.duration(hours=early_window_hours)).alias("start_window_72"),
            (pl.col("first_intubation_time") + pl.duration(hours=72) + pl.duration(hours=early_window_hours)).alias("end_window_72"),
            (pl.col("first_intubation_time") + pl.duration(hours=72)).alias("exact_72h_point")
        ])
        .filter(
            pl.col("charttime").is_between(pl.col("start_window_72"), pl.col("end_window_72"))
        )
        .with_columns([
            (pl.col("charttime") - pl.col("exact_72h_point")).dt.total_seconds().abs().alias("abs_diff_secs")
        ])
    )

    closest_72 = (
        result_72
        .sort(["hadm_id", "abs_diff_secs"])
        .group_by("hadm_id")
        .first()
        .select(["hadm_id", "valuenum"])
        .rename({"valuenum": f"{vital_name}_72h"})
    )

    # Join both results back into ventilation_df
    ventilation_df = (
        ventilation_df
        .join(closest_early, on="hadm_id", how="left")
        .join(closest_72, on="hadm_id", how="left")
    )

ventilation_df
# total_rows = ventilation_df.height
# null_counts = ventilation_df.null_count().to_dict(as_series=False)

# for col, count_list in null_counts.items():
#     count = count_list[0]  # unpack from list
#     percent = (count / total_rows) * 100 if total_rows > 0 else 0
#     print(f"{col}: {percent:.2f}% missing")

In [ ]:
ventilation_df = ventilation_df.with_columns([
    (
        0.098 * pl.col("resp_rate_4h") *
        (pl.col("tidal_volume_4h") / 1000) *
        (pl.col("peak_pressure_4h") - pl.col("peep_4h")) +
        pl.col("peep_4h")
    ).alias("mech_power_4h"),

    (
        0.098 * pl.col("resp_rate_72h") *
        (pl.col("tidal_volume_72h") / 1000) *
        (pl.col("peak_pressure_72h") - pl.col("peep_72h")) +
        pl.col("peep_72h")
    ).alias("mech_power_72h")
])
ventilation_df = ventilation_df.with_columns([
    (pl.col("cumulative_vent_duration").dt.total_seconds() / 86400).alias("cumulative_vent_days")
])
ventilation_df = ventilation_df.drop("cumulative_vent_duration")
ventilation_df

In [ ]:
icustays = pl.read_csv('datasets/KMIMIC/icustays.csv')
ventilation_df = ventilation_df.join(icustays[['subject_id', 'hadm_id']], on='hadm_id')
ventilation_df

In [ ]:
diagnoses_icd = pl.read_csv('datasets/KMIMIC/diagnoses_icd.csv')
b = diagnoses_icd.filter(
    pl.col("icd_code").str.starts_with("A40") |
    pl.col("icd_code").str.starts_with("A41") |
    pl.col("icd_code").str.starts_with("R65.2") |
    pl.col("icd_code").str.starts_with("R57.2")
)
a = ventilation_df.join(b[['hadm_id', 'icd_code']], on='hadm_id', how='left')
a = a.with_columns((a['icd_code'].is_not_null()).alias('sepsis_flag'))
a = a.drop('icd_code')
patients = pl.read_csv('datasets/KMIMIC/patients.csv')
patients = (
    patients.filter(pl.col("anchor_age").str.contains("years"))
      .with_columns(
          pl.col("anchor_age")
          .str.extract(r"(\d+)", 1)
          .cast(pl.Int32)
          .alias("age")
      )
      .filter(pl.col("age").is_not_null())
)
patients = patients.with_columns([
    (pl.col("sex") == "M").alias("gender_M"),
    (pl.col("sex") == "F").alias("gender_F")
])
a = a.join(patients[['age', 'subject_id', 'gender_M', 'gender_F']], on='subject_id', how='inner')
a

In [ ]:
a.write_csv('kmimic_final.csv')